# Mera Pupet Colab Server

This Colab notebook is for running Mera Pupet rendering server. It allows you to run Mera Pupet on your computer **without GPU**

# Install Setup

In [ ]:
!cd /content
!rm -rf *

In [ ]:
!git clone https://github.com/msafdarkhan/merapuppet.git

Cloning into 'avatarify'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 1323 (delta 18), reused 15 (delta 6), pack-reused 1283
Receiving objects: 100% (1323/1323), 5.61 MiB | 1.45 MiB/s, done.
Resolving deltas: 100% (843/843), done.


In [ ]:
cd avatarify

/content/avatarify


In [ ]:
!git clone https://github.com/msafdarkhan/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy pyyaml==5.1

Cloning into 'fomm'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 58.16 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (108/108), done.
     |████████████████████████████████| 276kB 3.9MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=605d6ecca99d72141d13149d9aa0c70c4aac5432d5db06aa7e4d630134c4a154
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!scripts/download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228M  100  228M    0     0  14.2M      0  0:00:16  0:00:16 --:--:-- 16.5M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [ ]:
# Download ngrok
!scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


# Run
Start here if the runtime was restarted after installation.

In [ ]:
cd /content/merapuppet

/content/merapuppet


In [ ]:
#!git pull origin

In [ ]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [ ]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [ ]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

This command should print lines if the worker is successfully started

In [ ]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root         157 33.5  2.2 2133656 294736 ?      S    17:22   0:02 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         171  0.0  1.2 2141852 160532 ?      Sl   17:23   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         174  0.0  1.1 2133656 157692 ?      S    17:23   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         175  0.0  1.2 2141852 160540 ?      Sl   17:23   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-po

# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [ ]:
# Paste your authtoken here in quotes
authtoken = ""

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [ ]:
# Set your region here in quotes
region = "ap"

In [ ]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [ ]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [ ]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

tcp://0.tcp.ap.ngrok.io:15584 -> 5558 [output]
tcp://0.tcp.ap.ngrok.io:13586 -> 5557 [input]
Tunnel opened


### [Optional] AWS proxy
Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command (substitute `run_mac.sh` with `run_windows.bat` or `run.sh`)
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [ ]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
When you run the cell below it will print a command. Run this command on your computer:

1. Open a terminal (in Windows open `Anaconda Prompt`);
2. Change working directory to the `avatarify` directory:</br>
* Windows (change `C:\path\to\avatarify` to your path)</br>
`cd C:\path\to\avatarify`</br></br>
* Mac/Linux (change `/path/to/avatarify` to your path)</br>
`cd /path/to/avatarify`
3. Copy-paste to the terminal the command below and run;
4. It can take some time to connect (usually up to 10 seconds). If the preview window doesn't appear in a minute or two, look for the errors above in this notebook and report in the [issues](https://github.com/alievk/avatarify/issues) or [Slack](https://join.slack.com/t/avatarify/shared_invite/zt-dyoqy8tc-~4U2ObQ6WoxuwSaWKKVOgg).

In [ ]:
print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Copy-paste to the terminal the command below and run (press Enter)

Mac:
./run_mac.sh --is-client --in-addr tcp://0.tcp.ap.ngrok.io:13586 --out-addr tcp://0.tcp.ap.ngrok.io:15584

Windows:
run_windows.bat --is-client --in-addr tcp://0.tcp.ap.ngrok.io:13586 --out-addr tcp://0.tcp.ap.ngrok.io:15584

Linux:
./run.sh --is-client --in-addr tcp://0.tcp.ap.ngrok.io:13586 --out-addr tcp://0.tcp.ap.ngrok.io:15584


# Logs

If something doesn't work as expected, please run the cells below and include the logs in your report.

In [ ]:
#@title
!cat ./var/log/cam_fomm.log | head -100

In [ ]:
#@title
!cat ./var/log/recv_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/send_worker.log | tail -100